In [1]:
import os
from dotenv import find_dotenv, load_dotenv

try:
    envPath = find_dotenv()

    if not envPath:
        raise ModuleNotFoundError(".env FILE NOT FOUND")
    
    if not load_dotenv(envPath):
        raise EnvironmentError("FAILED TO LOAD .env FILE")
    
    openai_api_key, openai_model = os.environ.get("OPENAI_API_KEY"), os.environ.get("OPENAI_MODEL")

    if not openai_api_key:
        raise ValueError('API KEY NOT FOUND')
    if not openai_model:
        raise ValueError('MODEL NOT FOUND')
except Exception as e:
    print(f"ERROR: {e}")
    openai_api_key, openai_model = None, None

In [2]:
from langgraph.graph import MessagesState, StateGraph, START, END
from langchain_core.messages import HumanMessage, SystemMessage, AnyMessage
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.prebuilt import tools_condition, ToolNode
from typing import Annotated, TypedDict
import operator
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI

In [8]:
def multiply(a: int, b: int) -> int:
    """Multiply Tool"""
    return a * b

def divide(a: int, b: int) -> int:
    """Divide Tool"""
    try:
        result = a / b

    except Exception as e:
        print(f"Error: {e}")
        result = 0

    return result

def add(a: int, b: int) -> int:
    """Addition Tool"""
    return a + b

def subtract(a: int, b: int) -> int:
    """Subtraction Tool"""
    return a - b

In [9]:
search = DuckDuckGoSearchRun()